# SEQ05 redux
## testing the fall 2018 sequence processing method on old samples
Because we are now using a targeted set of ~1020 contigs, we can trim, map and call SNPs on individual sequencing runs and delete the intermediate files when done.

## Prepare the directories for sequence storage

In [2]:
!ls /local/shared/pinsky_lab/sequencing/hiseq_2014_12_19_SEQ05/

clownfish-ddradseq-seq05-for-222-cycles-hau41adxx_1_read_1_passed_filter.fastq.gz
clownfish-ddradseq-seq05-for-222-cycles-hau41adxx_1_read_2_index_read_passed_filter.fastq.gz
clownfish-ddradseq-seq05-for-222-cycles-hau41adxx_2_read_1_passed_filter.fastq.gz
clownfish-ddradseq-seq05-for-222-cycles-hau41adxx_2_read_2_index_read_passed_filter.fastq.gz



## Go to the Princeton webpage to retrieve the seq data - took about 5 minutes for SEQ18 - 576 samples
https://htseq.princeton.edu/cgi-bin/login.pl?redirect_url=https://htseq.princeton.edu/cgi-bin/dashboard.pl **done**
* Click on the sequencing run of interest in the box on the left that says “Recently Entered Samples"
* In the box titled Sample Provenance, click on the link following "This library was utilized within the following output(s):” - repeat for each lane
* In the “Data and Statistics” box, in the bottom right corner is a green button that says “Download"
* click wget and then the copy button to get all files.  This will copy the wget commands for all files.
* paste in amphiprion **in the directory you made in the previous step**

## Make directories in personal workspace - 4 pools 

In [4]:
%%bash
cd /data/apcl
# mkdir 04seq
cd ./05seq/ 
# mkdir logs samples scripts
mkdir bcsplit 01Pool 02Pool 03Pool 04Pool 
cd bcsplit/
mkdir lane1 lane2

In [6]:
!cp -r ~/02-apcl-ddocent/05seq/logs /data/apcl/05seq/

In [7]:
!cp -r ~/02-apcl-ddocent/05seq/scripts /data/apcl/05seq/

In [8]:
!ls /data/apcl/05seq/logs

05seq-blacklist.txt  26process.out	names-24       rxstacks
05seq-loci.txt	     26process.out.tsv	names-25       rxstacks.out
24process.out	     27process.out	names-26       sstacks.out
24process.out.tsv    27process.out.tsv	names-27       stacks
25process.out	     barcodes		names-seq05    unpaired
25process.out.tsv    index-seq05	rxsstacks.out  ustacks-05.out


## Make sure index file is present
* see seq_proc.Rmd notebook in RStudio to generate index and names files for large batches of pools/samples

## Make sure barcodes file is present

## Run barcode splitter in lane1 folder and lane 2 folder - takes about 8 hours for 192 samples
* Closing the notebook closes this.  
* Paste into terminal window.
* started 12:30pm on Monday

cd /data/apcl/05seq/bcsplit/lane1
nohup /local/home/michelles/14_programs/paired_sequence_utils/barcode_splitter.py \
--bcfile ../../logs/index-seq05 --idxread 2 --suffix .fastq.gz \
/local/shared/pinsky_lab/sequencing/hiseq_2014_12_19_SEQ05/clownfish-ddradseq-seq05-for-222-cycles-hau41adxx_1_read_1_passed_filter.fastq.gz \
/local/shared/pinsky_lab/sequencing/hiseq_2014_12_19_SEQ05/clownfish-ddradseq-seq05-for-222-cycles-hau41adxx_1_read_2_index_read_passed_filter.fastq.gz &

cd /data/apcl/05seq/bcsplit/lane2
nohup /local/home/michelles/14_programs/paired_sequence_utils/barcode_splitter.py \
--bcfile ../../logs/index-seq05 --idxread 2 --suffix .fastq.gz \
/local/shared/pinsky_lab/sequencing/hiseq_2014_12_19_SEQ05/clownfish-ddradseq-seq05-for-222-cycles-hau41adxx_2_read_1_passed_filter.fastq.gz \
/local/shared/pinsky_lab/sequencing/hiseq_2014_12_19_SEQ05/clownfish-ddradseq-seq05-for-222-cycles-hau41adxx_2_read_2_index_read_passed_filter.fastq.gz &

## Check the output statistics in the nohup and move to logs with a rename

In [3]:
!head /data/apcl/05seq/bcsplit/lane1/nohup.out

line: P024	ACTTGA
line: P025	TAGCTT
line: P026	GGCTAC
line: P027	CTTGTA
P024	ACTTGA	22403790	16.32%
P025	TAGCTT	54671176	39.83%
P026	GGCTAC	24586195	17.91%
P027	CTTGTA	29205845	21.28%
unmatched	None	6386478	4.65%


In [4]:
!mv /data/apcl/05seq/bcsplit/lane1/nohup.out /data/apcl/05seq/logs/lane1.out

In [5]:
!head /data/apcl/05seq/bcsplit/lane2/nohup.out

line: P024	ACTTGA
line: P025	TAGCTT
line: P026	GGCTAC
line: P027	CTTGTA
P024	ACTTGA	22108464	16.35%
P025	TAGCTT	53965890	39.91%
P026	GGCTAC	24321226	17.99%
P027	CTTGTA	28819117	21.31%
unmatched	None	5993232	4.43%


In [6]:
!mv /data/apcl/05seq/bcsplit/lane2/nohup.out /data/apcl/05seq/logs/lane2.out

## Concatenate the results in the bcsplit folder - took about 1 minute

In [7]:
%%bash
cd /data/apcl/05seq/bcsplit/
cat ./lane1/P024-read-1.fastq.gz ./lane2/P024-read-1.fastq.gz > ../01Pool/P024.fastq.gz
cat ./lane1/P025-read-1.fastq.gz ./lane2/P025-read-1.fastq.gz > ../02Pool/P025.fastq.gz
cat ./lane1/P026-read-1.fastq.gz ./lane2/P026-read-1.fastq.gz > ../03Pool/P026.fastq.gz
cat ./lane1/P027-read-1.fastq.gz ./lane2/P027-read-1.fastq.gz > ../04Pool/P027.fastq.gz

## Run process radtags with script - takes ~2 hours for 192 samples
* Had to **nano the script** to point to the correct place and changed my email to gmail so I will get it on my phone (if it sends)
* If this notebook closes, these processes stop - **ended up pasting this into a terminal window** because I was afraid the network would drop and the process would stop

* Started at 6:42am on Tuesday
* finished between 7:45am and 9:15am on Tuesday

%%bash
cd /data/apcl/05seq/01Pool/
nohup ../scripts/24process.sh &
cd /data/apcl/05seq/02Pool/
nohup ../scripts/25process.sh &
cd /data/apcl/05seq/03Pool/
nohup ../scripts/26process.sh &
cd /data/apcl/05seq/04Pool/
nohup ../scripts/27process.sh &

## Move the process_radtags logs to the logs folder - wrote command lines for large batches in seq_proc.Rmd notebook and saved the lines in files called SEQXX-move_radlogs.sh 

The script did this for us so we don't have to

In [ ]:
#%%bash
#mv 01Pool/process_radtags.log ./logs/processP012.log
#mv 02Pool/process_radtags.log ./logs/processP013.log
#mv 03Pool/process_radtags.log ./logs/processP014.log
#mv 04Pool/process_radtags.log ./logs/processP015.log

In [1]:
!ls logs

05seq-blacklist.txt  26process.out	names-24       rxstacks
05seq-loci.txt	     26process.out.tsv	names-25       rxstacks.out
24process.out	     27process.out	names-26       sstacks.out
24process.out.tsv    27process.out.tsv	names-27       stacks
25process.out	     barcodes		names-seq05    unpaired
25process.out.tsv    index-seq05	rxsstacks.out  ustacks-05.out


## Run the readprocesslog.py script to convert the process log into a tsv that can be imported into the database through R
* This was done years ago, not doing now.

%%bash
cd ~/02-apcl-ddocent/04seq/01Pool/
readprocesslog.py ../logs/20process.out
cd ~/02-apcl-ddocent/04seq/02Pool/
readprocesslog.py ../logs/21process.out
cd ~/02-apcl-ddocent/04seq/03Pool/
readprocesslog.py ../logs/22process.out
cd ~/02-apcl-ddocent/04seq/04Pool/
readprocesslog.py ../logs/23process.out

## Move the newly created tsvs to the laptop with fetch 
* import them into the database using the R notebook script read_to_db.Rmd

## Rename the sample files
* recreate the names files using only the ligation id
* current names files have sample id included
* using the seq_proc.Rmd script in the methods repository > procedural notebooks

In [4]:
%%bash
cd /data/apcl/05seq/01Pool/
sh rename.for.dDocent_se_gz ../logs/names_24.tsv
mv APCL* /data/apcl/05seq/samples/
cd /data/apcl/05seq/02Pool/
sh rename.for.dDocent_se_gz ../logs/names_25.tsv
mv APCL* /data/apcl/05seq/samples/
cd /data/apcl/05seq/03Pool/
sh rename.for.dDocent_se_gz ../logs/names_26.tsv
mv APCL* /data/apcl/05seq/samples/
cd /data/apcl/05seq/04Pool/
sh rename.for.dDocent_se_gz ../logs/names_27.tsv
mv APCL* /data/apcl/05seq/samples/

APCL_L0878
TCTTAG
APCL_L0926
TCTTAG
APCL_L0974
TCTTAG
APCL_L1022
TCTTAG


## Once all of the samples have been moved, delete the pool directories

In [5]:
!rm -r /data/apcl/05seq/*Pool

## Identify regenotyped samples, cat and move them.

* using the R script cat_regenos.Rmd
* files were all cat'd during seq28

## Create a directory to trim, map, and call SNPs for seq03, 04, and 05 together 

In [6]:
!mkdir /data/apcl/03_04_05seq_snps

## Symlink the sample files from 03, 04, and 05 into the combined directory

In [8]:
!ln -s /data/apcl/03seq/samples/APCL_* /data/apcl/03_04_05seq_snps/
!ln -s /data/apcl/04seq/samples/APCL_* /data/apcl/`03_04_05seq_snps/
!ln -s /data/apcl/05seq/samples/APCL_* /data/apcl/03_04_05seq_snps/

ln: creating symbolic link `/data/apcl/03_04_05seq_snps/APCL_L0255.F.F.fq.gz': File exists
ln: creating symbolic link `/data/apcl/03_04_05seq_snps/APCL_L0255.F.fq.gz': File exists
ln: creating symbolic link `/data/apcl/03_04_05seq_snps/APCL_L0255.R1.fq.gz': File exists
ln: creating symbolic link `/data/apcl/03_04_05seq_snps/APCL_L0255-RG.bam': File exists
ln: creating symbolic link `/data/apcl/03_04_05seq_snps/APCL_L0255-RG.bam.bai': File exists
ln: creating symbolic link `/data/apcl/03_04_05seq_snps/APCL_L0256.F.F.fq.gz': File exists
ln: creating symbolic link `/data/apcl/03_04_05seq_snps/APCL_L0256.F.fq.gz': File exists
ln: creating symbolic link `/data/apcl/03_04_05seq_snps/APCL_L0256.R1.fq.gz': File exists
ln: creating symbolic link `/data/apcl/03_04_05seq_snps/APCL_L0256-RG.bam': File exists
ln: creating symbolic link `/data/apcl/03_04_05seq_snps/APCL_L0256-RG.bam.bai': File exists
ln: creating symbolic link `/data/apcl/03_04_05seq_snps/APCL_L0257.F.F.fq.gz': File exists
ln: creat

ln: creating symbolic link `/data/apcl/03_04_05seq_snps/APCL_L0639.F.fq.gz': File exists
ln: creating symbolic link `/data/apcl/03_04_05seq_snps/APCL_L0640.F.fq.gz': File exists
ln: creating symbolic link `/data/apcl/03_04_05seq_snps/APCL_L0641.F.fq.gz': File exists
ln: creating symbolic link `/data/apcl/03_04_05seq_snps/APCL_L0642.F.fq.gz': File exists
ln: creating symbolic link `/data/apcl/03_04_05seq_snps/APCL_L0643.F.fq.gz': File exists
ln: creating symbolic link `/data/apcl/03_04_05seq_snps/APCL_L0644.F.fq.gz': File exists
ln: creating symbolic link `/data/apcl/03_04_05seq_snps/APCL_L0645.F.fq.gz': File exists
ln: creating symbolic link `/data/apcl/03_04_05seq_snps/APCL_L0646.F.fq.gz': File exists
ln: creating symbolic link `/data/apcl/03_04_05seq_snps/APCL_L0647.F.fq.gz': File exists
ln: creating symbolic link `/data/apcl/03_04_05seq_snps/APCL_L0648.F.fq.gz': File exists
ln: creating symbolic link `/data/apcl/03_04_05seq_snps/APCL_L0649.F.fq.gz': File exists
ln: creating symbolic

ln: creating symbolic link `/data/apcl/03_04_05seq_snps/APCL_samples.sh': File exists


## Remove all symlinks and try again without extra files

In [9]:
!rm /data/apcl/03_04_05seq_snps/*

In [10]:
!ln -s /data/apcl/03seq/samples/APCL_*.F.fq.gz /data/apcl/03_04_05seq_snps/
!ln -s /data/apcl/04seq/samples/APCL_*.F.fq.gz /data/apcl/03_04_05seq_snps/
!ln -s /data/apcl/05seq/samples/APCL_*.F.fq.gz /data/apcl/03_04_05seq_snps/

## Copy reference.fasta over from jonsfiles

In [11]:
!cp /local/home/michelles/02-apcl-ddocent/jonsfiles/reference.fasta /data/apcl/03_04_05seq_snps/

## Trim, map , and call SNPs using dDocent
for this phase, dDocent will ask questions.  

* Are the number of samples correct?  

yes  

* Maximum number of processors to use for this analysis  

This depends on how many people are trying to use amphiprion right now.  The trim and map section isn't too intensive so it should be ok to use alot.  I used 30 for SEQ18 - 576 samples  

30  

* Maximum memory  

Again, consider who else is using the machine, for SEQ18 - 576 samples I used 150  

150  

* Quality trim?  

yes  

* Perform assembly?  

no - this is for creating the reference originally  

* Map reads?  

yes  

* Adjust default parameters  

default match score is 1, default mismatch is 4, gap penalty is 6 - I used defaults for all  

* call SNPs  

yes  

* enter email address to be notified when it is done running

In [12]:
%%bash
cd /data/apcl/03_04_05seq_snps/
dDocent

dDocent 2.5.5 

Contact jpuritz@uri.edu with any problems 

 
Checking for required software

All required software is installed!

dDocent run started Tue Oct 9 09:36:32 EDT 2018 

703 individuals are detected. Is this correct? Enter yes or no and press [ENTER]
Incorrect Input


/local/home/michelles/14_programs/dDocent/dDocent: line 84: [: 1.0-r82: integer expression expected
/local/home/michelles/14_programs/dDocent/dDocent: line 101: [: : integer expression expected


## 703 sounds like too many.  Are there duplicates?

In [14]:
!ls /data/apcl/03_04_05seq_snps/

APCL_L0255.F.F.fq.gz  APCL_L0394.F.fq.gz    APCL_L0792.F.fq.gz
APCL_L0255.F.fq.gz    APCL_L0395.F.F.fq.gz  APCL_L0793.F.fq.gz
APCL_L0256.F.F.fq.gz  APCL_L0395.F.fq.gz    APCL_L0794.F.fq.gz
APCL_L0256.F.fq.gz    APCL_L0397.F.F.fq.gz  APCL_L0795.F.fq.gz
APCL_L0257.F.F.fq.gz  APCL_L0397.F.fq.gz    APCL_L0796.F.fq.gz
APCL_L0257.F.fq.gz    APCL_L0398.F.F.fq.gz  APCL_L0797.F.fq.gz
APCL_L0258.F.F.fq.gz  APCL_L0398.F.fq.gz    APCL_L0798.F.fq.gz
APCL_L0258.F.fq.gz    APCL_L0400.F.F.fq.gz  APCL_L0799.F.fq.gz
APCL_L0259.F.F.fq.gz  APCL_L0400.F.fq.gz    APCL_L0800.F.fq.gz
APCL_L0259.F.fq.gz    APCL_L0401.F.F.fq.gz  APCL_L0801.F.fq.gz
APCL_L0260.F.F.fq.gz  APCL_L0401.F.fq.gz    APCL_L0802.F.fq.gz
APCL_L0260.F.fq.gz    APCL_L0402.F.F.fq.gz  APCL_L0803.F.fq.gz
APCL_L0261.F.F.fq.gz  APCL_L0402.F.fq.gz    APCL_L0804.F.fq.gz
APCL_L0261.F.fq.gz    APCL_L0403.F.F.fq.gz  APCL_L0805.F.fq.gz
APCL_L0262.F.F.fq.gz  APCL_L0403.F.fq.gz    APCL_L0806.F.fq.gz
APCL_L0262.F.fq.gz    APCL_L0405.F.F.fq.gz  APCL_L0807.

## Delete the .F.F files

In [15]:
!rm /data/apcl/03_04_05seq_snps/*.F.F.fq.gz

In [16]:
%%bash
cd /data/apcl/03_04_05seq_snps/
dDocent

dDocent 2.5.5 

Contact jpuritz@uri.edu with any problems 

 
Checking for required software

All required software is installed!

dDocent run started Tue Oct 9 09:38:36 EDT 2018 

540 individuals are detected. Is this correct? Enter yes or no and press [ENTER]
Incorrect Input


/local/home/michelles/14_programs/dDocent/dDocent: line 84: [: 1.0-r82: integer expression expected
/local/home/michelles/14_programs/dDocent/dDocent: line 101: [: : integer expression expected


## Combine TotalRawSNPs into master SNP doc for further analysis.